In [1]:
import numpy as np
import pandas as pd
import pickle
from scipy.special import softmax
import matplotlib.pyplot as plt
import os
import sklearn.metrics as metrics
from scipy import stats
import seaborn as sns



# Post-process model output 

In [2]:
def get_COBRA(HC_dir,HAR_dir):
    acts = ['brushing','combing','deodorant','drinking','facewash','feeding','glasses','shelf','RTT']
    ##############################
    ### model output - healthy ###
    ##############################
    confidence_hc = {}
    confidence_hc['sub'] = []
    confidence_hc['act'] = []
    confidence_hc['rep'] = []
    confidence_hc['prob'] = []
    confidence_hc['label'] = []
    for f in os.listdir(HC_dir):
        if 'label' not in f:
            sub = int(f.split('000')[1].split('_')[0])
            for a in acts:
                if a=='deodorant':
                    if 'deodrant' in f or 'deodorant' in f:
                        act=a
                elif a=='facewash':
                    if 'face' in f:
                        act=a
                else:
                    if a in f:
                        act=a                
            rep = f.split('.n')[0][-3:]
            prob = np.load(HC_dir+'/'+f)
            label = np.load(HC_dir+'/'+f.replace('.npy','_label.npy'))
            confidence_hc['sub'].append(sub)
            confidence_hc['act'].append(act)
            confidence_hc['rep'].append(rep)
            confidence_hc['prob'].append(prob)
            confidence_hc['label'].append(label)

    HC_df = pd.DataFrame(confidence_hc)
    HC_df['motion_prob'] = HC_df[['prob','label']].apply(lambda x: np.mean(x[0].max(axis=1)[x[1]<3]),axis=1)
    HC_df['non_motion_prob'] = HC_df[['prob','label']].apply(lambda x: np.mean(x[0].max(axis=1)[x[1]>2]),axis=1)
    HC_df['all_prob'] = HC_df[['prob','label']].apply(lambda x: np.mean(x[0].max(axis=1)),axis=1)
    
    #############################
    ### model output - stroke ###
    #############################
    confidence_har = {}
    confidence_har['sub'] = []
    confidence_har['act'] = []
    confidence_har['rep'] = []
    confidence_har['prob'] = []
    confidence_har['label'] = []
    for f in os.listdir(HAR_dir):
        if 'label' not in f:
            sub = int(f.split('000')[1].split('_')[0])
            for a in acts:
                if a=='deodorant':
                    if 'deodrant' in f or 'deodorant' in f:
                        act=a
                elif a=='facewash':
                    if 'face' in f:
                        act=a
                else:
                    if a in f:
                        act=a                
            rep = f.split('.n')[0][-3:]
            prob = np.load(HAR_dir+'/'+f)
            label = np.load(HAR_dir+'/'+f.replace('.npy','_label.npy'))
            confidence_har['sub'].append(sub)
            confidence_har['act'].append(act)
            confidence_har['rep'].append(rep)
            confidence_har['prob'].append(prob)
            confidence_har['label'].append(label)

    HAR_df = pd.DataFrame(confidence_har)
    HAR_df['motion_prob'] = HAR_df[['prob','label']].apply(lambda x: np.mean(x[0].max(axis=1)[x[0].argmax(axis=1)<3]),axis=1)
    HAR_df['non_motion_prob'] = HAR_df[['prob','label']].apply(lambda x: np.mean(x[0].max(axis=1)[x[0].argmax(axis=1)>2]),axis=1)
    HAR_df['all_prob'] = HAR_df[['prob','label']].apply(lambda x: np.mean(x[0].max(axis=1)),axis=1)

    return HC_df, HAR_df

### calculate COBRA acore for all video outputs

In [3]:
HC_df = None
HAR_df = None
HC_blurred_df = None
HAR_blurred_df = None


n_folds = 4
FM_file_path = '/gpfs/scratch/by2026/quantitativeRehabilitation/stroke_impairment.csv'
for i in range(1,5):
    HC_dir=f'/gpfs/scratch/by2026/quantitativeRehabilitation/extracted_logits/0906_all_HC_split{i}_best_err/processed_videos/test'
    HAR_dir=f'/gpfs/scratch/by2026/quantitativeRehabilitation/extracted_logits/0906_all_HC_split{i}_best_err/processed_videos/test_stroke'
    
    HC_df_fold,HAR_df_fold =get_COBRA(HC_dir,HAR_dir)
    
    if HC_df is None:
        HC_df = HC_df_fold
    else:
        HC_df = pd.concat([HC_df,HC_df_fold],ignore_index=True)
    
    if HAR_df is None:
        HAR_df = HAR_df_fold
    else:
        HAR_df = pd.concat([HAR_df,HAR_df_fold],ignore_index=True)
        
    HC_blurred_dir=f'/gpfs/scratch/by2026/quantitativeRehabilitation/extracted_logits/0906_all_HC_split{i}_best_err/processed_videos/test_blurry_healthy'
    HAR_blurred_dir=f'/gpfs/scratch/by2026/quantitativeRehabilitation/extracted_logits/0906_all_HC_split{i}_best_err/processed_videos/test_blurry'
    HC_df_fold,HAR_df_fold =get_COBRA(HC_blurred_dir,HAR_blurred_dir)
    
    if HC_blurred_df is None:
        HC_blurred_df = HC_df_fold
    else:
        HC_blurred_df = pd.concat([HC_blurred_df,HC_df_fold],ignore_index=True)
    
    if HAR_blurred_df is None:
        HAR_blurred_df = HAR_df_fold
    else:
        HAR_blurred_df = pd.concat([HAR_blurred_df,HAR_df_fold],ignore_index=True)
     


/gpfs/share/apps/anaconda3/gpu/5.2.0/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/gpfs/share/apps/anaconda3/gpu/5.2.0/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


In [4]:
HC_df.to_csv('HC_df.csv',index=False)
HAR_df.to_csv('HAR_df.csv',index=False)

HC_blurred_df.to_csv('HC_blurred_df.csv',index=False)
HAR_blurred_df.to_csv('HAR_blurred_df.csv',index=False)